# Práctica 3: Representación del conocimiento

SOLUCIÓN

El objetivo de esta práctica es aplicar los conceptos teóricos vistos en clase en el módulo de Representación de conocimiento. La práctica consta de un único notebook que se entregará en la tarea de entrega habilitada en el Campus  Virtual.

Cada consulta debe contener breves comentarios que expliquen cada tripleta. __No se valorarán consultas sin explicaciones__.

__Número de grupo: XX__

__Nombres de los estudiantes: XXX y XXX__

## Consultas SPARQL sobre Wikidata.

En esta práctica vamos a usar el [punto de acceso SPARQL](https://query.wikidata.org/) de Wikidata para contestar las preguntas que se formulan a continuación. Cada pregunta debe ser respondida realizando una única consulta SPARQL. Para cada una de las entidades recuperadas se mostrará __tanto su identificador como su etiqueta__ (nombre de la entidad en lenguaje natural).

Para cada una de las preguntas debes mostrar tanto la consulta como la respuesta obtenida. Si lo consideras necesario, puedes añadir celdas adicionales en formato _Markdown_ para explicar decisiones que hayas tomado al crear la consulta o cualquier otro dato que consideres interesante.

Para resolver estas consultas puedes usar __dos recursos que te recomendamos consultar__ son:

- [Este tutorial de SPARQL](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial).
- [Esta recopilación de ejemplos](https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples)

In [4]:
## Lo primero sería instalar en tu entorno de Python wdsparql
## Si no lo tienes instalado, puedes hacerlo desde el notebook
import sys

!{sys.executable} -m pip install wdsparql

# Asumismos instalado wdsparql
%load_ext autoreload
%load_ext wdsparql
%autoreload 2

### Ejemplo

Recuperar todas las instancias directas de la clase [Cabra (Q2934)](https://www.wikidata.org/wiki/Q2934) que aparecen en la base de conocimiento.

In [1]:
%%wdsparql
SELECT ?item ?itemLabel
WHERE
{
  ?item wdt:P31 wd:Q2934.  # instancias directas de la clase Cabra
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }  #traduce el identificador 
}

UsageError: Cell magic `%%wdsparql` not found.


### Consulta 1

[Stephen King (Q39829)](https://www.wikidata.org/wiki/Q39829) es un escritor estadounidense de novelas de terror, ficción sobrenatural, misterio, ciencia ficción y literatura fantástica.

Vamos a comenzar por averiguar su fecha y lugar de nacimiento (localidad y país al que pertenece la localidad en la actualidad).

In [31]:
%%wdsparql
SELECT ?birth ?location ?locationLabel
WHERE
{
    wd:Q39829 wdt:P569 ?birth;  #PUNTO y como indica para seguir buscando cosas dentro de wd:Q39829. Asi me ahorro poner otra vez wd:Q39829 wdt:P19 ?location
              wdt:P19 ?location.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    
}


,birth,location,locationLabel
0,1947-09-21T00:00:00Z,http://www.wikidata.org/entity/Q49201,Portland


### Consulta 2

A continuación vamos a averiguar todas las distintas profesiones (ocupaciones) que se le reconocen en la base de conocimiento. Queremos obtener los resultados ordenados alfabéticamente por el nombre de la profesión.

In [34]:
%%wdsparql
SELECT ?ocupacion ?ocupacionLabel
WHERE{
    wd:Q39829 wdt:P106 ?ocupacion
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?ocupacionLabel)

,ocupacion,ocupacionLabel
0,http://www.wikidata.org/entity/Q33999,actor
1,http://www.wikidata.org/entity/Q1086863,columnist
2,http://www.wikidata.org/entity/Q3455803,director
3,http://www.wikidata.org/entity/Q2526255,film director
4,http://www.wikidata.org/entity/Q1930187,journalist
5,http://www.wikidata.org/entity/Q6625963,novelist
6,http://www.wikidata.org/entity/Q18844224,science fiction writer
7,http://www.wikidata.org/entity/Q28389,screenwriter
8,http://www.wikidata.org/entity/Q37226,teacher
9,http://www.wikidata.org/entity/Q578109,television producer


### Consulta 3

De todas esas profesiones, ¿cuáles corresponden con subtipos de [Escritor (Q36180)](https://www.wikidata.org/wiki/Q36180)? Ten en cuenta que la jerarquía de tipos de escritores puede ser muy sofisticada.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Property_paths) te resulte útil.

In [12]:
%%wdsparql
SELECT ?ocupacion ?ocupacionLabel
WHERE{
    wd:Q39829 wdt:P106 ?ocupacion.
    ?ocupacion wdt:P279+ wd:Q36180. 
                  
              
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?ocupacionLabel)

,ocupacion,ocupacionLabel
0,http://www.wikidata.org/entity/Q1086863,columnist
1,http://www.wikidata.org/entity/Q6625963,novelist
2,http://www.wikidata.org/entity/Q18844224,science fiction writer
3,http://www.wikidata.org/entity/Q28389,screenwriter
4,http://www.wikidata.org/entity/Q36180,writer


Buscamos reiteradamente con la operacion * en cada ocupación (wdt:P106) de Sthepen King, si  es subclase de wd:Q36180 mediante la propierdad Subclass of (P279)

Si no añadimos el `+` a la relación `wdt:P279` muchas de las profesiones no se recuperan.

### Consulta 4

King se ha casado una vez, con Tabitha Spruce, con la que tiene varios hijos. Queremos conocer los nombres de esos hijos y sus fechas de nacimiento (si están disponibles). Los resultados deben aparecer ordenados cronológicamente.

In [27]:
%%wdsparql
SELECT ?hijos ?hijosLabel ?fechaNacimiento
WHERE{
     wd:Q39829 wdt:P40 ?hijos.
     ?hijos wdt:P25 wd:Q284287.
     OPTIONAL{?hijos wdt:P569 ?fechaNacimiento.} # No obligamos a que la fecha de nacimiento tome un valor
     SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC (?fechaNacimiento)


C:\Users\Itziar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\wdsparql\wdsparqlmagic.py:25: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  return pd.DataFrame(data, columns=columns).apply(pd.to_numeric, errors="ignore")


,hijos,hijosLabel,fechaNacimiento
0,http://www.wikidata.org/entity/Q19366533,Naomi King,None
1,http://www.wikidata.org/entity/Q370377,Joe Hill,1972-06-04T00:00:00Z
2,http://www.wikidata.org/entity/Q707117,Owen King,1977-02-21T00:00:00Z


Seleccionamos los hijos de Stephen King (wdt:P40) y en cada hijo buscamos el valor de la propiedad madre (P:25) y comparamos si es Tabitha King (Q284287). Después obtenemos la fecha de nacimiento de los hijos (P:569).

### Consulta 5

King ha recibido muchos premios a lo largo de su carrera. Queremos obtener la lista de premios y, para cada uno de ellos, la fecha en la que fue premiado (si está disponible) y la obra premiada (si está disponible). Los resultados se deben mostrar ordenados por fecha.

Para resolver esta consulta necesitarás acceder a los cualificadores de nodos sentencia y necesitarás entender los prefijos que usa Wikidata. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Qualifiers) te resulte útil.

In [30]:
%%wdsparql
SELECT ?premios ?premiosLabel ?fechaPremio ?obra ?obraLabel
WHERE{
     wd:Q39829 p:P166 ?nodo.
     ?nodo ps:P166 ?premios.
     ?nodo pq:P585 ?fechaPremio.
     OPTIONAL{?nodo pq:P1686 ?obra} 
     SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC (?fechaPremio)

C:\Users\Itziar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\wdsparql\wdsparqlmagic.py:25: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  return pd.DataFrame(data, columns=columns).apply(pd.to_numeric, errors="ignore")


,premios,premiosLabel,fechaPremio,obra,obraLabel
0,http://www.wikidata.org/entity/Q1056237,Hugo Award for Best Related Work,1982-01-01T00:00:00Z,http://www.wikidata.org/entity/Q526884,Danse Macabre
1,http://www.wikidata.org/entity/Q19339,O. Henry Award,1996-01-01T00:00:00Z,None,None
2,http://www.wikidata.org/entity/Q221178,Bram Stoker Award for Novel,1996-01-01T00:00:00Z,http://www.wikidata.org/entity/Q466288,The Green Mile
3,http://www.wikidata.org/entity/Q1027922,Grand Prix de l'Imaginaire,1997-01-01T00:00:00Z,http://www.wikidata.org/entity/Q526884,Danse Macabre
4,http://www.wikidata.org/entity/Q848963,Bram Stoker Awards,2000-01-01T00:00:00Z,http://www.wikidata.org/entity/Q438298,Riding the Bullet
5,http://www.wikidata.org/entity/Q3910464,Bram Stoker Award for Best Non-Fiction,2000-01-01T00:00:00Z,http://www.wikidata.org/entity/Q786363,On Writing
6,http://www.wikidata.org/entity/Q1027922,Grand Prix de l'Imaginaire,2002-01-01T00:00:00Z,None,None
7,http://www.wikidata.org/entity/Q848963,Bram Stoker Awards,2003-01-01T00:00:00Z,http://www.wikidata.org/entity/Q247364,The Dark Tower V: Wolves of the Calla
8,http://www.wikidata.org/entity/Q572316,National Book Award,2003-01-01T00:00:00Z,http://www.wikidata.org/entity/Q76981,Stephen King bibliography
9,http://www.wikidata.org/entity/Q221178,Bram Stoker Award for Novel,2006-01-01T00:00:00Z,http://www.wikidata.org/entity/Q957123,Lisey's Story


Una vez que hemos accedido a Stephen King , accedemos a los premios como si fuesen nodos y en cada uno de esos nodos accedemos a la información que contenga: fecha en la que se recivió el premio ( accesible por P585) y obra por la que fue recibido (accesible por P166). 

### Consulta 6

King es sin duda un autor prolífico. ¿Cuántas obras ha escrito a lo largo de su vida? Vamos a considerar sólo instancias directas de [literary work (Q7725634)](https://www.wikidata.org/wiki/Q7725634).

In [35]:
%%wdsparql
SELECT ?obras ?obrasLabel
WHERE{
    wd:Q39829 wdt:P800 ?obras.
    ?obras wdt:P31 wd:Q7725634
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

C:\Users\Itziar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\wdsparql\wdsparqlmagic.py:25: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  return pd.DataFrame(data, columns=columns).apply(pd.to_numeric, errors="ignore")


,obras,obrasLabel
0,http://www.wikidata.org/entity/Q149552,The Stand
1,http://www.wikidata.org/entity/Q239725,Black House
2,http://www.wikidata.org/entity/Q247364,The Dark Tower V: Wolves of the Calla
3,http://www.wikidata.org/entity/Q250123,Christine
4,http://www.wikidata.org/entity/Q302960,Bag of Bones
5,http://www.wikidata.org/entity/Q462454,The Talisman
6,http://www.wikidata.org/entity/Q464557,The Colorado Kid
7,http://www.wikidata.org/entity/Q466288,The Green Mile
8,http://www.wikidata.org/entity/Q466790,The Dead Zone
9,http://www.wikidata.org/entity/Q468121,Rose Madder


### Consulta 7

Aunque King es mundialmente conocido por sus obras de ficción ([Q8253](https://www.wikidata.org/wiki/Q8253)), es también autor de algunas obras menos conocidas de no-ficción ([Q213051](https://www.wikidata.org/wiki/Q213051)). Queremos saber cuáles son estas obras de no-ficción (o cualquiera de sus subgéneros) y su tipo. Los resultados se deben mostrar ordenados por nombre.

No pasa nada si en el resultado aparecen mezclados novelas, relatos, colecciones...

In [19]:
%%wdsparql
SELECT?obras ?obrasLabel ?genero ?generoLabel
WHERE{
    wd:Q7725634 wdt:P279* ?clase . #todas las clases a las que pertenece literary work y la propia literary work
    ?obras wdt:P31 ?clase;
           wdt:P50 wd:Q39829; # el autor debe de ser Stephen King
           wdt:P136 ?genero.  #guardamos el género de la obra
    ?genero wdt:P279* wd:Q213051. #ver que el género es subclase de non-fiction
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?obrasLabel)


C:\Users\Itziar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\wdsparql\wdsparqlmagic.py:25: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  return pd.DataFrame(data, columns=columns).apply(pd.to_numeric, errors="ignore")


,obras,obrasLabel,genero,generoLabel
0,http://www.wikidata.org/entity/Q526884,Danse Macabre,http://www.wikidata.org/entity/Q35760,essay
1,http://www.wikidata.org/entity/Q526884,Danse Macabre,http://www.wikidata.org/entity/Q35760,essay
2,http://www.wikidata.org/entity/Q16964977,Guns,http://www.wikidata.org/entity/Q213051,nonfiction
3,http://www.wikidata.org/entity/Q16385963,Head Down,http://www.wikidata.org/entity/Q35760,essay
4,http://www.wikidata.org/entity/Q786363,On Writing,http://www.wikidata.org/entity/Q112983,memoir
5,http://www.wikidata.org/entity/Q786363,On Writing,http://www.wikidata.org/entity/Q35760,essay
6,http://www.wikidata.org/entity/Q786363,On Writing,http://www.wikidata.org/entity/Q4184,autobiography


### Consulta 8

De todas las obras que ha publicado, queremos recuperar los nombres y fechas de publicación de aquellas escritas entre 1970 y 1980. Si alguna obra tiene varias fechas de publicación, se mostrará si alguna de ellas está dentro de ese periodo.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#FILTER) te resulte útil.

In [31]:
%%wdsparql
SELECT ?obras ?obrasLabel ?fecha
WHERE{
     wd:Q7725634 wdt:P279* ?clase . #todas las clases a las que pertenece literary work y la propia literary work
     ?obras wdt:P31 ?clase;
            wdt:P50 wd:Q39829;
            wdt:P577 ?fecha.
    FILTER(YEAR(?fecha) <= 1980 && YEAR(?fecha)>=1970).
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

HTTPError: 500 Server Error: Internal Server Error for url: https://query.wikidata.org/sparql?query=SELECT+%3Fobras+%3Ffechas+%3FobrasLabel+%3FfechasLabel%0AWHERE+%7B%0A++%3Fobras+wdt%3AP31%2Fwdt%3AP279%2A+wd%3AQ47461344.+%23+instancias+de+written+work+y+subclases+de+written+work%0A++%3Fobras+wdt%3AP50+wd%3AQ39829.+%23+el+autor+debe+de+ser+Stephen+King%0A++%0A++OPTIONAL+%7B%0A++++%3Fobras+wdt%3AP577+%3Ffechas.%0A++++FILTER%28%221970-01-01%22%5E%5Exsd%3AdateTime+%3C%3D+%3Ffechas+%26%26+%3Ffechas+%3C+%221980-01-01%22%5E%5Exsd%3AdateTime%29%0A++%7D%0A++%0A++SERVICE+wikibase%3Alabel+%7B+bd%3AserviceParam+wikibase%3Alanguage+%22en%22.+%7D%0A%7D%0A%23ORDER+BY+ASC%28%3FobrasLabel%29%0A&format=json

### Consulta 9

Wikidata contiene información actualizada que en ocasiones incluye futuros lanzamientos u obras planificadas. Vamos a dejar por un momento a King y vamos a considerar la obra de otro famoso autor, George R. R. Martin.

[George R. R. Martin (Q181677)](https://www.wikidata.org/wiki/Q181677) es un escritor de fantasía estadounidense conocido sobre todo por su saga "Canción de hielo y fuego". Martin tiene inconclusa la saga y queremos saber si hay alguna información sobre los libros pendientes. Estos serán sus obras sin fecha de publicación asociada. ¿Aparecen? Si lo hacen, ¿cuántas y cuáles son sus títulos? ¿Hay alguna obra pendiente que no sea de esa saga?

In [27]:
%%wdsparql
SELECT ?obras ?obrasLabel ?fecha ?saga ?sagaLabel
WHERE{
     wd:Q7725634 wdt:P279* ?clase . #todas las clases a las que pertenece literary work y la propia literary work
     ?obras wdt:P31 ?clase; #creamos instancias de cada clase
            wdt:P50 wd:Q181677. #obligamos que el autor sea George R.R Martin
            
    OPTIONAL{?obras wdt:P577 ?fecha}
    OPTIONAL{?obras wdt:P179 ?saga}
    FILTER (!BOUND(?fecha)) #filtramos para que solo muestre las que no tengan la fecha definida
    
    
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

HTTPError: 500 Server Error: Internal Server Error for url: https://query.wikidata.org/sparql?query=SELECT+DISTINCT+%3Fobras+%3FobrasLabel+%3Fpublication+%3FpublicationLabel%0AWHERE+%7B%0A++%3Fobras+wdt%3AP31%2Fwdt%3AP279%2A+wd%3AQ15621286%3B+%0A+++++++++wdt%3AP50+wd%3AQ181677%3B++%23+George+R.+R.+Martin+como+autor%0A+++++++++wdt%3AP577+%3FpublicationDate.+%23+Fecha+de+publicaci%C3%B3n%0A+++++++++SERVICE+wikibase%3Alabel+%7B+bd%3AserviceParam+wikibase%3Alanguage+%22en%22.+%7D%0A++%23FILTER+NOT+EXISTS+%7B+%3Fwork+wdt%3AP577+%3FpublicationDate.+%7D+%23+Obras+sin+fecha+de+publicaci%C3%B3n%0A%7D%0A&format=json

### Consulta 10

Volvemos de nuevo a King. De todas las obras que ha publicado (independientemente de su tipo y género), queremos recuperar los nombres y fechas de publicación de aquellas escritas en coautoría, así como los nombnres de estos coautores.

In [6]:
%%wdsparql
SELECT ?obras ?obrasLabel ?coautor ?coautorLabel
WHERE{
     wd:Q7725634 wdt:P279* ?clase . #todas las clases a las que pertenece literary work y la propia literary work
     ?obras wdt:P31 ?clase; #creamos instancias de cada clase
            wdt:P50 wd:Q39829; #obligamos que el autor sea stephen king
            wdt:P50 ?coautor. #guardamos los valores de la propieda autor
    FILTER(?coautor != wd:Q39829) ¿

    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

### Consulta 11

¿Con cuántos coautores diferentes ha trabajado King, independientemente del tipo y género de la obra?

In [5]:
%%wdsparql
SELECT ?coautor ?coautorLabel
WHERE{
     wd:Q7725634 wdt:P279* ?clase . #todas las clases a las que pertenece literary work y la propia literary work
     ?obras wdt:P31 ?clase; #creamos instancias de cada clase
            wdt:P50 wd:Q39829; #obligamos que uno de los  autores sea stephen king
            wdt:P50 ?coautor. #guardamos los valores de la propiedad autor
    FILTER(?coautor != wd:Q39829) #nos quedamos con aquellos en los que no aparezca sthepen king
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?coautor ?coautorLabel

C:\Users\Itziar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\wdsparql\wdsparqlmagic.py:25: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  return pd.DataFrame(data, columns=columns).apply(pd.to_numeric, errors="ignore")


,coautor,coautorLabel
0,http://www.wikidata.org/entity/Q364189,Peter Straub
1,http://www.wikidata.org/entity/Q1585875,Stewart O'Nan
2,http://www.wikidata.org/entity/Q325130,Richard Matheson
3,http://www.wikidata.org/entity/Q346993,Joe Hill
4,http://www.wikidata.org/entity/Q7324736,Richard Chizmar
5,http://www.wikidata.org/entity/Q707117,Owen King


### Consulta 12

¿Qué otros personajes famosos nacieron el mismo año que King y tienen su misma nacionalidad? Para cada uno de ellos muestra su nombre y ocupación. Muestra los resultados ordenados por nombre, fecha de nacimiento y ocupación.

### Consulta 13

¿Qué otros personajes famosos incluyen King en su nombre completo (name in native language, P1559)? Para cada uno de ellos muestra su nombre, fecha de nacimiento, ocupación y nacionalidad. Muestra los resultados ordenados por nombre, fecha de nacimiento y ocupación.

In [ ]:
%%wdsparql
SELECT ?name ?nameLabel ?date ?dateLabel ?occupation ?occupationLabel ?nationality ?nationalityLabel
WHERE{
    ?person wdt:P31 wd:Q5; #buscamos instancias de persona
            wdt:P1559 ?name; #guardamos el nombre, fecha de nacimiento, nacionalidad y ocupación
            wdt:P569  ?date;
            wdt:P106  ?occupation;
            wdt:P27   ?nationality.
    FILTER(CONTAINS(?name, 'King'))#miramos que el nombre contenga la palabra "King"
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". } 
}
ORDER BY ASC(?name) ASC(?date) ASC(?occupation)

### Consulta 14

¿Cuántos asertos hay sobre King en Wikidata? Ten en cuenta que puede aparecer tanto como sujeto como objeto de una tripleta.

In [19]:
%%wdsparql
SELECT (COUNT(*) AS ?count)
WHERE {
  {
    ?subject ?predicate wd:Q39829.
  }
  UNION
  {
    wd:Q8891 ?predicate ?object.
  }
}

C:\Users\Itziar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\wdsparql\wdsparqlmagic.py:25: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  return pd.DataFrame(data, columns=columns).apply(pd.to_numeric, errors="ignore")


,count
0,2081


__Fecha de las consultas: 21 de abril de 2022__